In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
from torch.utils.data import DataLoader
from model import mhVAE
from train import train_model,pretrain_model
from util import setup_seed
from scipy.optimize import linear_sum_assignment
from preprocess import  geneSelection,my_normalize

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score,adjusted_mutual_info_score
from sklearn.preprocessing import LabelEncoder
import scipy.io as sio
import anndata as ad
import scipy
import scipy as sp

import h5py
import sklearn

/home/shiyinan/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
### read datasets
path = '/data/'

ctfile = '10x1kpbmc_label.mat'


celltype_data = sio.loadmat(path+ctfile)
celltype=celltype_data['y']
data = celltype_data['X']
rna=data[0][1][:,:,0]
pro = data[0][0][:,:,0]


adata1=ad.AnnData(rna)
adata2=ad.AnnData(pro)



/tmp/ipykernel_3590899/3343788062.py:15: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata1=ad.AnnData(rna)
/tmp/ipykernel_3590899/3343788062.py:16: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata2=ad.AnnData(pro)


In [ ]:


setup_seed(42)
data1_=geneSelection(adata1.X, threshold=0, atleast=10, yoffset=.02, xoffset=5, decay=1.5, n=500, plot=False, markers=None, genes=None, figsize=(6,3.5),markeroffsets=None, labelsize=10, alpha=1, verbose=1)
data1=adata1[:,data1_]
data1 = my_normalize(data1,size_factors=True, normalize_input=True, logtrans_input=True)

data2 = my_normalize(adata2,size_factors=True, normalize_input=True, logtrans_input=True)

citeseq1= np.concatenate([data1.X,data2.X], axis=1)
print(citeseq1.shape)

nfeatures_rna = data1.shape[1]
nfeatures_pro = data2.shape[1]
## parameters
batch_size = 128
epochs_per_cycle =1
epochs = epochs_per_cycle*100
lr = 0.01
z_dim = 100
hidden_rna2 =200
hidden_pro2 =200

citeseq=pd.DataFrame(citeseq1)
train_data=citeseq.to_numpy(dtype=np.float32)

# load data
train_transformed_dataset = train_data
train_dl = DataLoader(train_transformed_dataset, batch_size=batch_size,shuffle=False, num_workers=0,drop_last=False)


Chosen offset: 0.68
(713, 517)


In [ ]:

model = mhVAE(num_features=[nfeatures_rna,nfeatures_pro], num_hidden_features=[hidden_rna2,hidden_pro2], z_dim=z_dim).cuda()

In [5]:
pretrain_model(model,train_dl,lr,epochs=50)

100%|██████████| 50/50 [00:04<00:00, 11.44it/s]


In [ ]:
##cuda = True if torch.cuda.is_available() else False
model,history,embedding=train_model(model, train_dl, lr, epochs=100)

100%|██████████| 100/100 [00:05<00:00, 17.24it/s]


In [ ]:
kmeans = KMeans(n_clusters = 5,n_init=20)
consensus_labels = kmeans.fit_predict(embedding[0])
consensus_labels=kmeans.labels_
#print(consensus_labels)

In [ ]:

label_encoder = LabelEncoder()
true_labels_numeric = label_encoder.fit_transform(celltype)
consensus_labels_series = pd.Series(consensus_labels)
consensus_labels_encoded = label_encoder.fit_transform(consensus_labels_series)

ari = adjusted_rand_score(true_labels_numeric, consensus_labels_encoded)
nmi = normalized_mutual_info_score(true_labels_numeric, consensus_labels_encoded)
ami = adjusted_mutual_info_score(true_labels_numeric, consensus_labels_encoded)

def cluster_acc(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1  
    row_ind, col_ind = linear_sum_assignment(w.max() - w) 
    return sum([w[i, j] for i, j in zip(row_ind, col_ind)]) * 1.0 / y_pred.size
acc=cluster_acc(true_labels_numeric, consensus_labels_encoded)

print(true_labels_numeric)
print(consensus_labels_encoded)
print("ARI:", ari)
print("NMI:", nmi)
print("AMI:", ami)
print("ACC:",acc)

[4 0 0 0 0 2 2 2 0 0 2 0 0 0 4 2 2 2 2 1 2 0 1 0 2 3 1 4 0 0 1 4 0 3 1 0 2
 4 1 3 0 0 2 0 0 4 0 0 0 4 4 1 3 0 2 2 0 3 1 4 1 0 0 0 4 0 2 1 2 3 1 0 1 3
 2 0 2 0 0 0 0 2 3 3 1 3 1 2 4 4 1 3 2 3 0 0 4 1 0 0 1 0 0 2 0 3 0 3 0 2 3
 1 4 0 0 3 0 3 1 4 3 3 3 1 0 4 1 0 0 2 1 1 0 0 3 2 3 0 3 2 0 1 0 0 4 0 2 2
 2 2 4 3 2 3 0 0 0 1 2 2 2 1 1 0 3 2 2 3 3 3 0 3 0 2 4 2 4 1 2 4 0 0 1 0 2
 0 1 0 0 4 3 3 3 1 1 4 0 1 1 1 4 1 1 0 1 3 2 3 0 3 2 3 0 3 2 2 1 0 1 2 3 2
 0 1 1 1 2 3 2 4 0 1 1 2 0 1 0 3 4 3 1 4 0 4 1 1 2 0 0 0 0 3 4 2 3 2 2 3 3
 0 0 0 0 3 1 0 0 0 0 2 1 1 0 2 1 0 3 3 3 3 1 3 2 1 0 0 4 3 0 1 0 1 1 0 3 0
 3 0 0 2 0 3 2 0 4 1 2 1 0 1 1 0 3 3 0 1 1 4 0 1 0 0 2 1 0 0 0 3 1 1 3 3 3
 3 3 3 0 1 3 2 3 3 1 2 3 3 0 1 0 0 0 0 4 2 1 2 0 0 4 2 3 0 0 2 4 1 0 4 1 0
 1 2 0 0 0 3 2 2 1 0 2 3 1 2 0 2 0 1 1 3 0 2 2 3 0 0 0 0 0 4 1 0 4 4 0 2 1
 0 0 3 3 3 2 1 4 0 2 0 0 1 1 0 0 3 2 0 4 4 0 3 3 0 1 0 1 1 2 0 0 1 0 0 4 3
 4 2 3 4 0 1 0 1 4 3 2 2 0 0 2 0 1 4 0 2 1 4 0 2 1 0 1 4 1 0 1 0 0 1 2 0 1
 2 1 1 1 4 0 3 4 0 2 3 4 

/home/shiyinan/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
